In [ ]:
# pip install pymysql
# pip list        # pip를 이용해 설치한 패키지 목록 확인

In [6]:
import pymysql

# pymysql 모듈 설치 확인
print(pymysql.version_info)

# DB연동을 위한 설정 정보
config = {
    'host' : '127.0.0.1',       # localhost 주소
    'user'  : 'scott',
    'password' : 'tiger',
    'database' : 'work',
    'port' : 3306,
    'charset' : 'utf8',
    'use_unicode' : True
}

try :   
    ## DB 연동 객체 생성
    # 1) 직접 객체 정보 입력
    # conn = pymysql.connect(host = '127.0.0.1', user='scott',....)
    # 2) 사전에 설정된 정보를 적용
    conn = pymysql.connect(**config)   
    # *list -> list형태로 차례로 들어감 / **dict -> dict형태로 키=밸류 값이 차례로 들어감
    
    ## SQL문 실행 객체 - 커서 객체 (cursor)
    cursor = conn.cursor()
    
    ## 테이블 조회
    sql = "SHOW tables;"
    cursor.execute(sql)
    tables = cursor.fetchall()
    
    ## 테이블 유무 확인
    if tables :
        print('table 있음 : ', tables)
    else :
        print('테이블이 없습니다 !')
    
    
except Exception as e :
    print('DB연동 에러 발생 !',e)
    conn.rollback()     # INSERT, UPDATE, DELETE 시 사용
finally :
    cursor.close()
    conn.close()

(1, 4, 0, 'final', 0)
table 있음 :  (('goods',),)


In [40]:
## 연습 : Mysql CRUD 동작
# 1. 기존의 goods 테이블을 삭제
# 2. python 코드를 이용하여 DB 연결 객체를 생성
# 3. DB 안에 goods라는 테이블 생성
# 4. 생성된 goods 테이블에 레코드 추가 작성 - input을 이용하여 code, name, su, dan을 입력받아 추가
# 5. 전체 목록 보기 (전체 목록을 출력한 후에 전체 목록 개수도 출력)
# 6. 상품명 조회 - 상품명을 입력받아 해당 상품명에 따른 레코드를 출력)

import pymysql

config = {
    'host' : '127.0.0.1',
    'user'  : 'scott',
    'password' : 'tiger',
    'database' : 'work',
    'port' : 3306,
    'charset' : 'utf8',
    'use_unicode' : True
}

try :
    # 2. python 코드를 이용하여 DB 연결 객체를 생성
    conn = pymysql.connect(**config)
    cursor = conn.cursor()
    
    # 1. 기존의 goods 테이블을 삭제
    # sql = "DROP table if exists work.goods;"
    # cursor.execute(sql)
    # print('goods 테이블이 삭제되었습니다')
    
    # 3. DB 안에 goods라는 테이블 생성
    sql = """create table if not exists goods
            (code int primary key, name varchar(20) not null, 
            su int default 0, dan int default 0);"""
    cursor.execute(sql)
    
    # 4. 생성된 goods 테이블에 레코드 추가 작성
    chk = 1
    while chk != 0 :
        code = int(input('제품 코드를 입력하세요 : '))
        name = input('제품명을 입력하세요 : ')
        su = int(input('제품 수량을 입력하세요 : '))
        dan = int(input('제품 단가를 입력하세요 : '))
        
        sql = f"INSERT INTO goods VALUES ({code}, '{name}', {su}, {dan});"
        cursor.execute(sql)
        conn.commit()
        
        chk = int(input('계속 입력하시겠습니까?(1.계속/0.종료) : '))
    
    sql = "SELECT * FROM goods"
    cursor.execute(sql)
    tot_list = cursor.fetchall()
    print('총 개수 :',len(tot_list))
    for x in tot_list :
        print("%d   %s   %d   %d"%x)
    
    search = input('상품명을 입력하세요 : ')
    sql = f"SELECT * FROM goods WHERE name like '%{search}%';"      # 정보의 일부만 알고 있다면 like %str%
    cursor.execute(sql)
    product = cursor.fetchall()
    for product in product :
        print(f'찾으신 상품 정보 - 코드 : {product[0]}, 제품명 : {product[1]}, 수량 : {product[2]}, 단가 : {product[3]}')   

except Exception as e :
    print('오류 발생 !',e)
    conn.rollback()
finally :
    cursor.close()
    conn.close()   
    

총 개수 : 5
1   냉장고   5   990000
2   스타일러   8   490000
3   컴퓨터   9   1220000
4   의자   13   229000
6   태블릿PC   11   499000
찾으신 상품 정보 - 코드 : 6, 제품명 : 태블릿PC, 수량 : 11, 단가 : 499000


In [ ]:
# 위에서 만든 예제를 이용하여 다음과 같은 메뉴로 동작하는 DB프로그램을 작성
# 조건 1. 각 기능을 함수로 구현
# 조건 2. 반복 기능을 사용하여 구현하기(while)
'''
--- 메뉴 ---
1. 상품 등록
2. 상품 조회(전체)
3. 상품 조회(상품명으로 조회)
4. 상품 수정(코드로)
5. 상품 삭제(코드로)
6. 프로그램 종료
'''


def reg(cursor, conn) :
    sql = '''create table if not exists goods
            (code int primary key, name varchar(20) not null, su int default 0, dan int default 0)'''
    cursor.execute(sql)
    
    code = int(input('제품 코드를 입력하세요 : '))
    name = input('제품명을 입력하세요 : ')
    su = int(input('제품 수량을 입력하세요 : '))
    dan = int(input('제품 단가를 입력하세요 : '))
    
    sql = f"INSERT INTO goods VALUES ({code}, '{name}', {su}, {dan})"
    cursor.execute(sql)
    conn.commit()

def viewAll(cursor) :
    sql = "SELECT * FROM goods"
    cursor.execute(sql)
    tot_list = cursor.fetchall()
    print('총 개수 :',len(tot_list))
    for product in tot_list :
        print(f"코드 : {product[0]}, 제품명 : {product[1]}, 수량 : {product[2]}, 단가 : {product[3]}")

def viewName(cursor) :
    search = input('상품명을 입력하세요 : ')
    sql = f"SELECT * FROM goods WHERE name like '%{search}%'"      # 정보의 일부만 알고 있다면 like %str%
    cursor.execute(sql)
    product = cursor.fetchall()
    for product in product :
        print(f'찾으신 상품 정보 - 코드 : {product[0]}, 제품명 : {product[1]}, 수량 : {product[2]}, 단가 : {product[3]}')

def modify(cursor, conn) :
    code = input('수정할 코드를 입력하세요 : ')
    sql = f"SELECT * FROM goods WHERE code = {code}" 
    cursor.execute(sql)
    product = cursor.fetchone()
    print(f"코드 : {product[0]}, 1. 제품명 : {product[1]}, 2. 수량 : {product[2]}, 3. 단가 : {product[3]}")
    
    num = int(input('수정하실 카테고리를 선택하세요(숫자) : '))
    if num == 1 : 
        new = input('변경하실 제품명 입력 : ')
        sql = f"UPDATE goods SET name = '{new}' WHERE code = {code}"
    elif num == 2 : 
        new = int(input('변경하실 재고수량 입력 : '))
        sql = f"UPDATE goods SET su = {new} WHERE code = {code}"
    elif num == 3 : 
        new = int(input('변경하실 단가 입력 : '))
        sql = f"UPDATE goods SET dan = {new} WHERE code = {code}"
    cursor.execute(sql)
    conn.commit()
    sql = f"SELECT * FROM goods WHERE code = {code}" 
    cursor.execute(sql)
    product = cursor.fetchone()
    print(f"변경하신 내용 - 코드 : {product[0]}, 1. 제품명 : {product[1]}, 2. 수량 : {product[2]}, 3. 단가 : {product[3]}")
    
def delete(cursor, conn) :
    code = input('삭제할 코드를 입력하세요 : ')
    sql = f"SELECT * FROM goods WHERE code = {code}" 
    cursor.execute(sql)
    product = cursor.fetchone()
    print(f"코드 : {product[0]}, 1. 제품명 : {product[1]}, 2. 수량 : {product[2]}, 3. 단가 : {product[3]}")
    chk = input('정말 삭제하시겠습니까 ? (y/n) : ')
    if chk == 'y' :
        sql = f"DELETE FROM goods WHERE code = {code}"
    cursor.execute(sql)
    conn.commit()
    
    
    
import pymysql
config = {
    'host' : '127.0.0.1',
    'user'  : 'scott',
    'password' : 'tiger',
    'database' : 'work',
    'port' : 3306,
    'charset' : 'utf8',
    'use_unicode' : True
}
try:
    conn = pymysql.connect(**config)
    cursor = conn.cursor()
    
    while True :
        sel = int(input("""--- 메뉴 ---
                        1. 상품 등록
                        2. 상품 조회(전체)
                        3. 상품 조회(상품명으로 조회)
                        4. 상품 수정(코드로)
                        5. 상품 삭제(코드로)
                        6. 프로그램 종료
                        ===> """))
        if sel == 1 : reg(cursor, conn)
        elif sel == 2 : viewAll(cursor)
        elif sel == 3 : viewName(cursor)
        elif sel == 4 : modify(cursor, conn)
        elif sel == 5 : delete(cursor, conn)
        elif sel == 6 : break    
    
except Exception as e :
    print('오류 발생 !',e)
finally :
    cursor.close()
    conn.close()


In [ ]:
# 쌤이 하신 답
### 위 문제를 이용하여 다음과 같은 메뉴로 동작하는 DB프로그램을 작성

# -----메뉴-----
# 1. 상품 등록
# 2. 상품 조회(전체)
# 3. 상품 조회(상품명으로 조회)
# 4. 상품 수정(코드로..)
# 5. 상품 삭제(코드...)
# 0. 프로그램 종료

# 조건 1> 각 기능을 함수로 구현하세요. 
# 조건 2> 반복 기능을 사용하여 구현하세요(while을 이용한 무한 반복)
#        
import pymysql
import os
import sys

config = {              ## DB연동을 위한 환경변수
    'host': '192.168.149.1',    # 각자의 접속 DB 주소, 127.0.0.1
    'user': 'scott',
    'password': 'tiger',
    'database': 'work',
    'port':3306,
    'charset':'utf8',
    'use_unicode':True
}

def CreateTable():
    try:
        import pymysql
        # 연결 객체 생성
        conn = pymysql.connect(**config)      
        # SQL 동작을 위한 객체 cursor
        cursor = conn.cursor()
        # table 생성
        sql ="""create table if not exists goods(
            code int primary key,
            name varchar(30) not null,
            su int default 0,
            dan int default 0
        )"""
        cursor.execute(sql)
        conn.commit()
    except Exception as e:
        print("table 생성 실패 : ",e)
        conn.rollback()
    finally:
        cursor.close()
        conn.close()

def Insert(code,name,su,dan):
    try:
        conn = pymysql.connect(**config)      
        cursor = conn.cursor()
        sql = f"insert into goods values({code},'{name}',{su},{dan})"
        cursor.execute(sql)
        conn.commit()
    except Exception as e:
        print("Insert 실패 : ",e)
        conn.rollback()
    finally:
        cursor.close()
        conn.close()
        
def SelectAll():
    try:
        conn = pymysql.connect(**config)      
        cursor = conn.cursor()
        sql = "select * from goods"
        cursor.execute(sql)
        rows = cursor.fetchall()
        print("code\tname\t수량\t단가")
        for row in rows:
            print("{}\t{}\t{}\t{}".format(row[0],row[1],row[2],row[3]))
        print("총 상품 수량 : ",len(rows))
    except Exception as e:
        print("전체조회 실패 : ",e)
    finally:
        cursor.close()
        conn.close()


def SelectOne(name):
    try:
        conn = pymysql.connect(**config)      
        cursor = conn.cursor()
        sql = f"select * from goods where name like '%{name}%'"
        cursor.execute(sql)
        rows = cursor.fetchall()
        if rows:
            print("code\tname\t수량\t단가")
            for row in rows:
                print("{}\t{}\t{}\t{}".format(row[0],row[1],row[2],row[3]))
        else:
            print("해당 상품이 없습니다.")
        
    except Exception as e:
        print("개별 조회 실패 : ",e)
        conn.rollback()
    finally:
        cursor.close()
        conn.close()


def Update(code):
    try:
        conn = pymysql.connect(**config)      
        cursor = conn.cursor()
        name = input("변경할 상품명을 입력 : ")
        su = int(input("변경할 수량을 입력 : "))
        dan = int(input("변경할 단가 입력 : "))
        sql = f"update goods set name={name}, su={su}, dan={dan} where code={code}"
        cursor.execute(sql)
        conn.commit()
        
    except Exception as e:
        print("Update 실패 : ",e)
        conn.rollback()
    finally:
        cursor.close()
        conn.close()


def Delete(code):
    try:
        conn = pymysql.connect(**config)      
        cursor = conn.cursor()
        sql = f"delete from goods where code={code}"
        del_sel = input("정말삭제하겠습니까?(y/N) : ")
        if del_sel.lower() == 'y':
            cursor.execute(sql)
            conn.commit()
        else: 
            print("삭제를 취소합니다.")        
    except Exception as e:
        print("삭제 실패 : ",e)
        conn.rollback()
    finally:
        cursor.close()
        conn.close()


while True:
    os.system('cls')
    print("-----메뉴-----")
    print("1. 상품 등록")
    print("2. 상품 조회(전체)")
    print("3. 상품 조회(상품명으로 조회)")
    print("4. 상품 수정")
    print("5. 상품 삭제")
    print("0. 프로그램 종료")
    
    CreateTable()
    
    sel = input("메뉴를 선택하세요 : ")
    match sel:
        case '1':
            print("레코드 추가 정보를 입력하세요:")
            code = int(input("코드 입력 : "))
            name = input("상품명 입력 : ")
            su = int(input("수량 입력 : "))
            dan = int(input("단가 입력 : "))
            Insert(code, name, su, dan)
            
        case '2':
            print("전체 상품 조회")
            SelectAll()
        case '3':
            print("개별 상품 조회")
            name = input("상품명을 입력하세요 : ")
            SelectOne(name)
        case '4':
            print("상품 정보 수정")
            code = int(input("변경할 상품 코드 입력 : "))
            Update(code)
        case '5':
            print("상품 정보 삭제")
            code = int(input("삭제할 상품 코드 입력 : "))
            Delete(code)
        case '0':
            print("프로그램을 종료합니다.")
            os.system("pause")
            sys.exit(0)
            
        case _:
            print("메뉴 선택을 잘못했습니다.")
            os.system("pause")
    


In [ ]:
# 위에서 만들어진 함수를 class로 구성
'''
--- 메뉴 ---
1. 상품 등록
2. 상품 조회(전체)
3. 상품 조회(상품명으로 조회 / 코드 입력 시 코드로 조회)
4. 상품 수정(코드로)
5. 상품 삭제(코드로)
6. 프로그램 종료
'''
import pymysql
import os

class Exam :    
    def __init__(self, config) :
        self.conn = pymysql.connect(**config)
        self.cursor = self.conn.cursor()
        
    def reg(self) :
        sql = '''create table if not exists goods
            (code int primary key, name varchar(20) not null, su int default 0, dan int default 0)'''
        self.cursor.execute(sql)
        
        code = int(input('제품 코드를 입력하세요 : '))
        name = input('제품명을 입력하세요 : ')
        su = int(input('제품 수량을 입력하세요 : '))
        dan = int(input('제품 단가를 입력하세요 : '))
        
        sql = f"INSERT INTO goods VALUES ({code}, '{name}', {su}, {dan})"
        self.cursor.execute(sql)
        self.conn.commit()

    def viewAll(self) :
        sql = "SELECT * FROM goods"
        self.cursor.execute(sql)
        tot_list = self.cursor.fetchall()
        print('총 개수 :',len(tot_list))
        for product in tot_list :
            print(f"코드 : {product[0]}, 제품명 : {product[1]}, 수량 : {product[2]}, 단가 : {product[3]}")


    def viewName(self) :
        num = int(input('개별 상품 조회 방법 선택 - [1]상품명으로 검색 [2] 제품코드로 검색 : '))
        search = input('검색어 입력 : ')
        if num == 1 : sql = f"SELECT * FROM goods WHERE name like '%{search}%'"      # 정보의 일부만 알고 있다면 like %str%
        elif num == 2 : sql = f"SELECT * FROM goods WHERE code = {search}"
        self.cursor.execute(sql)
        product = self.cursor.fetchall()
        for product in product :
            print(f'찾으신 상품 정보 - 코드 : {product[0]}, 제품명 : {product[1]}, 수량 : {product[2]}, 단가 : {product[3]}')


    def modify(self) :
        code = input('수정할 코드를 입력하세요 : ')
        sql = f"SELECT * FROM goods WHERE code = {code}" 
        self.cursor.execute(sql)
        product = self.cursor.fetchone()
        print(f"코드 : {product[0]}, 1. 제품명 : {product[1]}, 2. 수량 : {product[2]}, 3. 단가 : {product[3]}")
        
        num = int(input('수정하실 카테고리를 선택하세요(숫자) : '))
        if num == 1 : 
            new = input('변경하실 제품명 입력 : ')
            sql = f"UPDATE goods SET name = '{new}' WHERE code = {code}"
        elif num == 2 : 
            new = int(input('변경하실 재고수량 입력 : '))
            sql = f"UPDATE goods SET su = {new} WHERE code = {code}"
        elif num == 3 : 
            new = int(input('변경하실 단가 입력 : '))
            sql = f"UPDATE goods SET dan = {new} WHERE code = {code}"
        self.cursor.execute(sql)
        self.conn.commit()
        
        sql = f"SELECT * FROM goods WHERE code = {code}" 
        self.cursor.execute(sql)
        product = self.cursor.fetchone()
        print(f"변경하신 내용 - 코드 : {product[0]}, 1. 제품명 : {product[1]}, 2. 수량 : {product[2]}, 3. 단가 : {product[3]}")
        
        
    def delete(self) :
        code = int(input('삭제할 코드를 입력하세요 : '))
        sql = f"SELECT * FROM goods WHERE code = {code}" 
        self.cursor.execute(sql)
        product = self.cursor.fetchone()
        print(f"코드 : {product[0]}, [1]제품명 : {product[1]}, [2]수량 : {product[2]}, [3]단가 : {product[3]}")
        chk = input('정말 삭제하시겠습니까 ? (y/n) : ')
        if chk == 'y' :
            sql = f"DELETE FROM goods WHERE code = {code}"
        self.cursor.execute(sql)
        self.conn.commit()
        
# === 메인 ===
import pymysql
config = {
    'host' : '127.0.0.1',
    'user'  : 'scott',
    'password' : 'tiger',
    'database' : 'work',
    'port' : 3306,
    'charset' : 'utf8',
    'use_unicode' : True
}
try:    
    pos = Exam(config)    
    while True :
        os.system('cls')
        print("-----메뉴-----")
        print("1. 상품 등록")
        print("2. 상품 조회(전체)")
        print("3. 상품 조회(상품명으로 조회)")
        print("4. 상품 수정")
        print("5. 상품 삭제")
        print("0. 프로그램 종료")
        sel = int(input('메뉴를 선택하세요 : '))
        if sel == 1 : 
            pos.reg()
            os.system('pause')
        elif sel == 2 : 
            pos.viewAll()
            os.system('pause')
        elif sel == 3 : 
            pos.viewName()
            os.system('pause')
        elif sel == 4 : 
            pos.modify()
            os.system('pause')
        elif sel == 5 : 
            pos.delete()
            os.system('pause')
        elif sel == 0 : 
            print('프로그램 종료 !')
            break
except Exception as e :
    print('오류 발생 !',e)
finally :
    pos.cursor.close()
    pos.conn.close()
